In [127]:
import pandas as pd
from fredlib import openFredGraph
import os
import numpy as np

In [100]:
dev_df = pd.read_pickle('./data/dev.pkl')
dev_df.describe()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg
count,1217,1217,974,1217,1217,1217,1216,1215,1214,1217,1217
unique,1217,1217,787,1216,438,1212,636,5,795,1217,1212
top,34656,A baby died at an unnamed medical facility be...,"March 17, 2015",The largest study of its kind found new eviden...,,Beta-Propiolactone is in vaccines and is known...,,true,Health News,[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'gentamicin sulfate', 'type': 'subje..."
freq,1,1,13,2,272,3,573,629,160,1,3


In [101]:
g = openFredGraph('./data/fred/dev/claims/6_claim.rdf')

In [102]:
dev_df.iloc[5]['claim']

'School closures will have little impact on COVID-19 control, review finds.'

In [103]:
len(g.getInfoEdges())

41

In [104]:
g.getInfoEdges()

{('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#remove_1',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Remove'): <fredlib.FredEdge at 0x2c3312899d0>,
 ('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#remove_1',
  'http://www.ontologydesignpatterns.org/ont/vn/abox/role/Theme',
  'http://www.ontologydesignpatterns.org/ont/fred/domain.owl#dam_1'): <fredlib.FredEdge at 0x2c33128b550>,
 ('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#remove_1',
  'http://www.ontologydesignpatterns.org/ont/vn/abox/role/Agent',
  'http://www.ontologydesignpatterns.org/ont/fred/domain.owl#fine_1'): <fredlib.FredEdge at 0x2c331357490>,
 ('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#threaten_1',
  'http://www.ontologydesignpatterns.org/ont/vn/abox/role/Experiencer',
  'http://www.ontologydesignpatterns.org/ont/fred/domain.owl#beaver_1'): <fredlib.FredEdge at 0x2c3312cc250>,
 ('http://www.ontolo

In [105]:
def clean_ontogolical_string(ontological_string):
    if '#' in ontological_string:
        if '_' in ontological_string:
            return ontological_string[ontological_string.find('#')+1:ontological_string.find('_')]
        else:
            return ontological_string[ontological_string.find('#')+1:]
    else:
        ontological_string =  ontological_string.split('/')[-1]
        if '_' in ontological_string:
            return ontological_string[:ontological_string.find('_')]
        else:
            return ontological_string

def get_RDF_triples_from_graph(edges):
    rdf_triples = []
    for triple in edges:
       
        subject = clean_ontogolical_string(triple[0]).lower()
        predicate = clean_ontogolical_string(triple[1])
        object = clean_ontogolical_string(triple[2]).lower()

        if subject==object:
            continue
    
        rdf_triple = (subject, predicate, object)
        rdf_triples.append(rdf_triple)
        
        # This is an invalid graph
        if 'runtime error' in object:
            return None

    return rdf_triples

rdf_triples = get_RDF_triples_from_graph(g.getInfoEdges())
print(rdf_triples)

[('remove', 'Theme', 'dam'), ('remove', 'Agent', 'fine'), ('threaten', 'Experiencer', 'beaver'), ('threaten', 'Cause', 'state'), ('fail', 'Theme', 'remove'), ('fail', 'Agent', 'fine'), ('michigan', 'type', 'place'), ('fine', 'type', 'dayfine'), ('beaver', 'type', 'localbeaver'), ('hasdatavalue', 'type', 'datatypeproperty'), ('michigan', 'type', 'administrativearea'), ('remove', 'subClassOf', 'event'), ('threaten', 'subClassOf', 'event'), ('fail', 'subClassOf', 'event'), ('dayfine', 'subClassOf', 'fine'), ('localbeaver', 'subClassOf', 'beaver'), ('day', 'hasDeterminer', 'a'), ('state', 'stateOf', 'michigan'), ('dam', 'damOf', 'thing'), ('day', 'hasQuantifier', '10%2c000'), ('day', 'associatedWith', 'fine'), ('beaver', 'hasQuantifier', 'multiple'), ('beaver', 'with', '10000'), ('dollar', 'hasDataValue', '10000'), ('beaver', 'hasQuality', 'local'), ('state', 'hasDeterminer', 'the'), ('dollar', 'per', 'fine'), ('dayfine', 'associatedWith', 'day'), ('dam', 'hasQuantifier', 'multiple')]


In [107]:
def capture_FRED_RDF_triples(folder_path, df, claim=True):
    files = os.listdir(folder_path)

    for index in range(dev_df.shape[0]):
        if claim:
            filename = f"{index}_claim.rdf"
        else:
            filename = f"{index}_main_text.rdf"

        if filename in files:
            file_path = os.path.join(folder_path, filename)
            try:
                g = openFredGraph(file_path)
                if 'claim' in filename:
                    rdf_triples = get_RDF_triples_from_graph(g.getInfoEdges())
                    if rdf_triples is not None:
                        df.at[index, 'FRED_claim_kg'] = rdf_triples
                else:
                    rdf_triples = get_RDF_triples_from_graph(g.getInfoEdges())
                    if rdf_triples is not None:
                        df.at[index, 'FRED_main_text_kg'] = rdf_triples
            except Exception as e:
                continue

            if index%100 == 0:
                print(f'Processed {index} files....')

    return df

dev_df = capture_FRED_RDF_triples('./data/fred/dev/claims/',dev_df)

Processed 200 files....
Processed 300 files....
Processed 400 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '1400000000.0'


Processed 700 files....
Processed 800 files....
Processed 900 files....
Processed 1100 files....
Processed 1200 files....


In [114]:
dev_df = capture_FRED_RDF_triples('./data/fred/dev/main_texts/',dev_df, claim=False)

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '208.'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '

Processed 700 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '1250000.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 

Processed 1200 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '1.9'


In [115]:
dev_df.describe()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg
count,1217,1217,974,1217,1217,1217,1216,1215,1214,1217,1217,1221,1221
unique,1217,1217,787,1216,438,1212,636,5,795,1217,1212,495,297
top,34656,A baby died at an unnamed medical facility be...,"March 17, 2015",The largest study of its kind found new eviden...,,Beta-Propiolactone is in vaccines and is known...,,true,Health News,[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'gentamicin sulfate', 'type': 'subje...",nan,nan
freq,1,1,13,2,272,3,573,629,160,1,3,727,913


In [125]:
dev_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",nan,nan
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...",nan,"[(report, Agent, ksnt), (report, Theme, find),..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",nan,nan
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",nan,nan
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",nan,nan


In [146]:
def supplant_missing_fred_kgs(row):
    
    if str(row['FRED_claim_kg'])=='nan':
        row['FRED_claim_kg'] = row['claim_kg']
    else:        
        triples = []
        for triple in row['FRED_claim_kg']:
            triples.append({'head':triple[0], 'type':triple[1], 'tail':triple[2]})
        
        row['FRED_claim_kg'] = triples

    if str(row['FRED_main_text_kg'])=='nan':
        row['FRED_main_text_kg'] = row['main_text_kg']
    else:
        triples = []
        for triple in row['FRED_main_text_kg']:
            triples.append({'head':triple[0], 'type':triple[1], 'tail':triple[2]})

        row['FRED_main_text_kg'] = triples

    return row

dev_df = dev_df.apply(supplant_missing_fred_kgs, axis=1)

In [147]:
dev_df.head(-1)

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc..."
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'report', 'type': 'Agent', 'tail': '..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...","[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'..."
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...","[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,9002,Sylentis announces the results of tivanisiran ...,"May 10, 2018",This news release focuses on a study of a new ...,"Earle Holland,Harold J. DeMonaco, MS,Kathlyn S...",There is no discussion of the potential cost o...,https://www.healthnewsreview.org/wp-content/up...,mixture,"dry eye,Pharmamar","[{'head': 'tivanisiran', 'type': 'medical cond...","[{'head': 'tivanisiran', 'type': 'medical cond...","[{'head': 'tivanisiran', 'type': 'medical cond...","[{'head': 'tivanisiran', 'type': 'medical cond..."
1220,4459,US regulators clear path for genetically modif...,NaN,U.S. regulators on Friday gave the green light...,Candice Choi,The Food and Drug Administration said it lifte...,https://www.ap.org/press-releases/2018/ap-hhmi...,true,"Genetics, Health, Fish, Salmon, North America,...","[{'head': 'genetically modified salmon', 'type...","[{'head': 'AquaBounty', 'type': 'headquarters ...","[{'head': 'genetically modified salmon', 'type...","[{'head': 'AquaBounty', 'type': 'headquarters ..."
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'head': 'topic', 'type': 'at', 'tail': '2017...","[{'head': 'find', 'type': 'in', 'tail': '17119..."
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'head': 'go', 'type': 'Theme', 'tail': 'comm...",NaN


In [149]:
dev_df = dev_df.dropna(thresh=11)
dev_df.describe()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg
count,1216,1216,973,1216,1216,1216,1216,1215,1214,1216,1216,1216,1216
unique,1216,1216,786,1215,437,1211,636,5,795,1216,1211,1216,1203
top,34656,A baby died at an unnamed medical facility be...,"March 17, 2015",The largest study of its kind found new eviden...,,Beta-Propiolactone is in vaccines and is known...,,true,Health News,[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'gentamicin sulfate', 'type': 'subje...",[{'head': 'A baby died at an unnamed medical f...,[]
freq,1,1,13,2,272,3,573,629,160,1,3,1,13


In [150]:
dev_df.to_pickle('./data/fred/dev/dev_fred.pkl')

In [151]:
test_df = pd.read_pickle('./data/test.pkl')
test_df.describe()

,Unnamed: 0,claim_id
count,1235.0000,1235.000000
mean,617.0000,17467.723077
std,356.6581,12347.916661
min,0.0000,37.000000
25%,308.5000,7573.000000
50%,617.0000,11516.000000
75%,925.5000,28851.500000
max,1234.0000,42236.000000


In [152]:
test_df = capture_FRED_RDF_triples('./data/fred/test/claims/',test_df)

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '128500000.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with bas

Processed 200 files....
Processed 300 files....
Processed 400 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '57400000000.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with b

Processed 500 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '3400000000000.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with

...

Processed 700 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '.0006'


Processed 800 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '3700000.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 

Processed 1000 files....
Processed 1200 files....


In [154]:
test_df.head()

,Unnamed: 0,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg
0,0,33456,A mother revealed to her child in a letter aft...,"November 6, 2011",The one-eyed mother story expounds upon two mo...,David Mikkelson,"In April 2005, we spotted a tearjerker on the ...",,false,Glurge Gallery,"[{'head': 'mother', 'type': 'child', 'tail': '...","[{'head': 'My mom only had one eye', 'type': '...",nan
1,1,2542,Study says too many Americans still drink too ...,"February 25, 2013","On any given day in the United States, 18 perc...",,That means the great majority of Americans sta...,http://bit.ly/X1NVtW,true,Health News,"[{'head': 'drink too much', 'type': 'facet of'...","[{'head': 'Patricia Guenther', 'type': 'occupa...","[(drink, Patient, much), (drink, Agent, study)..."
2,2,26678,Viral image Says 80% of novel coronavirus case...,"March 13, 2020",The website Information is Beautiful published...,Paul Specht,"Amid the spread of the novel coronavirus, many...",https://www.facebook.com/informationisbeautifu...,true,"Facebook Fact-checks, Coronavirus, Viral image,","[{'head': 'Mild', 'type': 'subclass of', 'tail...",[{'head': 'Center for Disease Control and Prev...,nan
3,3,40705,An email says that 9-year old Craig Shergold o...,"March 16, 2015",Send greeting or business cards to cancer vict...,Rich Buhler & Staff,Craig Shergold is real and in 1989...,https://www.reddit.com/submit?url=https%3A%2F%...,false,"Inspirational, Pleas","[{'head': 'Craig Shergold', 'type': 'place of ...","[{'head': 'Craig Shergold', 'type': 'date of b...","[(die, dieOf, cancer), (die, Theme, craig), (s..."
4,4,35718,"Employees at a Five Guys restaurant in Daphne,...","July 15, 2020","What's undetermined: As of this writing, Five ...",Dan MacGuill,"In July 2020, amid a new wave of nationwide pr...",,unproven,Law Enforcement,"[{'head': 'Five Guys', 'type': 'location of fo...","[{'head': 'Jubilee Square', 'type': 'located i...","[(refuse, agent, employee), (refuse, theme, se..."


In [155]:
test_df = capture_FRED_RDF_triples('./data/fred/test/main_texts/',test_df, claim=False)

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '10.6'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '

Processed 200 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '3.5'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '0

Processed 300 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '0.64'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '

...

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '0.6'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '1

Processed 700 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '11200000000.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with b

Processed 1000 files....


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '2.1'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '1

In [161]:
test_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg
0,33456,A mother revealed to her child in a letter aft...,"November 6, 2011",The one-eyed mother story expounds upon two mo...,David Mikkelson,"In April 2005, we spotted a tearjerker on the ...",,false,Glurge Gallery,"[{'head': 'mother', 'type': 'child', 'tail': '...","[{'head': 'My mom only had one eye', 'type': '...",nan,nan
1,2542,Study says too many Americans still drink too ...,"February 25, 2013","On any given day in the United States, 18 perc...",,That means the great majority of Americans sta...,http://bit.ly/X1NVtW,true,Health News,"[{'head': 'drink too much', 'type': 'facet of'...","[{'head': 'Patricia Guenther', 'type': 'occupa...","[(drink, Patient, much), (drink, Agent, study)...",nan
2,26678,Viral image Says 80% of novel coronavirus case...,"March 13, 2020",The website Information is Beautiful published...,Paul Specht,"Amid the spread of the novel coronavirus, many...",https://www.facebook.com/informationisbeautifu...,true,"Facebook Fact-checks, Coronavirus, Viral image,","[{'head': 'Mild', 'type': 'subclass of', 'tail...",[{'head': 'Center for Disease Control and Prev...,nan,nan
3,40705,An email says that 9-year old Craig Shergold o...,"March 16, 2015",Send greeting or business cards to cancer vict...,Rich Buhler & Staff,Craig Shergold is real and in 1989...,https://www.reddit.com/submit?url=https%3A%2F%...,false,"Inspirational, Pleas","[{'head': 'Craig Shergold', 'type': 'place of ...","[{'head': 'Craig Shergold', 'type': 'date of b...","[(die, dieOf, cancer), (die, Theme, craig), (s...",nan
4,35718,"Employees at a Five Guys restaurant in Daphne,...","July 15, 2020","What's undetermined: As of this writing, Five ...",Dan MacGuill,"In July 2020, amid a new wave of nationwide pr...",,unproven,Law Enforcement,"[{'head': 'Five Guys', 'type': 'location of fo...","[{'head': 'Jubilee Square', 'type': 'located i...","[(refuse, agent, employee), (refuse, theme, se...","[(event, hasDeterminer, the), (event, hasQuant..."


In [159]:
test_df = test_df.loc[:, ~test_df.columns.str.contains('^Unnamed')]

In [163]:
test_df = test_df.apply(supplant_missing_fred_kgs, axis=1)

In [164]:
test_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg
0,33456,A mother revealed to her child in a letter aft...,"November 6, 2011",The one-eyed mother story expounds upon two mo...,David Mikkelson,"In April 2005, we spotted a tearjerker on the ...",,false,Glurge Gallery,"[{'head': 'mother', 'type': 'child', 'tail': '...","[{'head': 'My mom only had one eye', 'type': '...","[{'head': 'mother', 'type': 'child', 'tail': '...","[{'head': 'My mom only had one eye', 'type': '..."
1,2542,Study says too many Americans still drink too ...,"February 25, 2013","On any given day in the United States, 18 perc...",,That means the great majority of Americans sta...,http://bit.ly/X1NVtW,true,Health News,"[{'head': 'drink too much', 'type': 'facet of'...","[{'head': 'Patricia Guenther', 'type': 'occupa...","[{'head': 'drink', 'type': 'Patient', 'tail': ...","[{'head': 'Patricia Guenther', 'type': 'occupa..."
2,26678,Viral image Says 80% of novel coronavirus case...,"March 13, 2020",The website Information is Beautiful published...,Paul Specht,"Amid the spread of the novel coronavirus, many...",https://www.facebook.com/informationisbeautifu...,true,"Facebook Fact-checks, Coronavirus, Viral image,","[{'head': 'Mild', 'type': 'subclass of', 'tail...",[{'head': 'Center for Disease Control and Prev...,"[{'head': 'Mild', 'type': 'subclass of', 'tail...",[{'head': 'Center for Disease Control and Prev...
3,40705,An email says that 9-year old Craig Shergold o...,"March 16, 2015",Send greeting or business cards to cancer vict...,Rich Buhler & Staff,Craig Shergold is real and in 1989...,https://www.reddit.com/submit?url=https%3A%2F%...,false,"Inspirational, Pleas","[{'head': 'Craig Shergold', 'type': 'place of ...","[{'head': 'Craig Shergold', 'type': 'date of b...","[{'head': 'die', 'type': 'dieOf', 'tail': 'can...","[{'head': 'Craig Shergold', 'type': 'date of b..."
4,35718,"Employees at a Five Guys restaurant in Daphne,...","July 15, 2020","What's undetermined: As of this writing, Five ...",Dan MacGuill,"In July 2020, amid a new wave of nationwide pr...",,unproven,Law Enforcement,"[{'head': 'Five Guys', 'type': 'location of fo...","[{'head': 'Jubilee Square', 'type': 'located i...","[{'head': 'refuse', 'type': 'agent', 'tail': '...","[{'head': 'event', 'type': 'hasDeterminer', 't..."


In [165]:
test_df = test_df.dropna(thresh=11)
test_df.describe()

,claim_id
count,1235.000000
mean,17467.723077
std,12347.916661
min,37.000000
25%,7573.000000
50%,11516.000000
75%,28851.500000
max,42236.000000


In [166]:
test_df.to_pickle('./data/fred/test/test_fred.pkl')